In [2]:
import numpy as np
import pandas as pd
import os
from sklearn import manifold
import umap

In [3]:
#the seris are of different lenght. split base on 2000 records
times_length = 2000

#loading the csv files
condition_path = os.path.join("C:\\Users\\Shamir\\PycharmProjects\\future eye\\condition\\condition_{}.csv")
control_path = os.path.join("C:\\Users\\Shamir\\PycharmProjects\\future eye\\control\\control_{}.csv")


In [4]:
condition_raw = [np.array(pd.read_csv(condition_path.format(x))['activity'])for x in range(1, 24)]
control_raw = [np.array(pd.read_csv(control_path.format(x))['activity'])for x in range(1, 33)]

In [5]:
truncate = lambda series: series[:times_length * (len(series) // times_length)]

In [6]:
condition = np.concatenate([truncate(series).reshape(-1, times_length) for series in condition_raw])
control = np.concatenate([truncate(series).reshape(-1, times_length) for series in control_raw])

In [7]:
# fixing the structure of te data and scaling down the data.
control = np.log(control + 1)
condition = np.log(condition + 1)
scale = max(control.max(), condition.max())
control = control / scale
condition = condition / scale

In [8]:
print("Saving processed data to")
print("condition_{}".format(times_length),"and","control_{}".format(times_length))
np.save("condition_{}".format(times_length), condition)
np.save("control_{}".format(times_length), control)

Saving processed data to
condition_2000 and control_2000


In [9]:
#UMAP Embedding

#Loadint the Saved Data 
condition = np.load("condition_{}.npy".format(times_length))
control = np.load("control_{}.npy".format(times_length))
#umap embedding on condition
umap_cond = umap.UMAP().fit_transform(condition)
#umap embedding on control
umap_cont = umap.UMAP().fit_transform(control)

In [10]:
#saving the embedding
np.save("condition_{}_umap".format(times_length), umap_cond)
np.save("control_{}_umap".format(times_length), umap_cont)

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from time import time

In [14]:
#defining train and validation size and rate of radomization
train = 652 / 752
validation = 100 / 652
random = np.random.randint(1, 2**16)

condition = np.load(os.path.join("condition_{}_umap.npy".format(times_length)))
control = np.load(os.path.join("control_{}_umap.npy".format(times_length)))

In [15]:
#splitting the data as train and validation
X = np.concatenate((condition, control), axis=0)
y = to_categorical(np.array([0] * len(condition) + [1] * len(control)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train, random_state=random)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation, random_state=random)

In [32]:
#training with a simple deeplearning classifier
model = Sequential()
model.add(Dense(4, input_dim=2, activation='relu'))
model.add(Dense(2, activation = 'softmax'))
model.add(Dense(2, input_dim=times_length, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

history = model.fit(
    X_train,
    y_train,
    batch_size=30,
    epochs=100,
    callbacks=[EarlyStopping(patience=10)],
    validation_split=0.1)

result = model.predict(X_val)
ypred = np.argmax(result, axis=1)
ytrue = y_val[:, 1]
print("Accuracy:", np.sum(ypred == ytrue) / 100)


Train on 496 samples, validate on 56 samples
Epoch 1/100
496/496 [==============================] - 1s 2ms/sample - loss: 0.8829 - acc: 0.5121 - val_loss: 0.7919 - val_acc: 0.6071
Epoch 2/100
496/496 [==============================] - 0s 95us/sample - loss: 0.8560 - acc: 0.5282 - val_loss: 0.7666 - val_acc: 0.6071
Epoch 3/100
496/496 [==============================] - 0s 105us/sample - loss: 0.8304 - acc: 0.5282 - val_loss: 0.7417 - val_acc: 0.6071
Epoch 4/100
496/496 [==============================] - 0s 100us/sample - loss: 0.8012 - acc: 0.5343 - val_loss: 0.7137 - val_acc: 0.6429
Epoch 5/100
496/496 [==============================] - 0s 95us/sample - loss: 0.7685 - acc: 0.6230 - val_loss: 0.6845 - val_acc: 0.6964
Epoch 6/100
496/496 [==============================] - 0s 97us/sample - loss: 0.7377 - acc: 0.6351 - val_loss: 0.6627 - val_acc: 0.6964
Epoch 7/100
496/496 [==============================] - 0s 102us/sample - loss: 0.7171 - acc: 0.6351 - val_loss: 0.6493 - val_acc: 0.6964
E